# Holography Fitting - Robustness Test
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyLab

Here we calculate the repeatability of the singular and binocular holography fitting using ML on the Simons Observatorry Large Aperture Telescope. 

In [ ]:
import numpy as np
import sys
import os
sys.path.append('/home/chesmore/Desktop/Code/holosim_paper/package/holosim-ml')
import tele_geo as tg
import ap_field as af
import pan_mod as pm
import optics_analyze as oa
import far_field as ff
import ap_fitting as afit
%matplotlib inline
import sklearn
from sklearn.linear_model import LinearRegression
import pickle
from matplotlib.ticker import MaxNLocator
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

font_manager.fontManager.addfont(
    "/home/chesmore/.local/share/fonts/times-new-roman.ttf"
)
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams["font.size"] = 28
plt.rcParams["image.cmap"] = "magma"
plt.style.use("ggplot")
plt.rcParams["axes.unicode_minus"] = False
save = 0

# Singular and Binocular positions
rx_x = np.array([0, 0, 0])
rx_z = np.array([-600 * (3 / 2), 600 * (3 / 2), 0])
el = np.array([oa.el_offset(-600 * (3 / 2)), oa.el_offset(600 * (3 / 2)), 0])
az = np.array([0, 0, 0])
shift_A = ["y", oa.sh_z(rx_z[0])]
shift_B = ["y", oa.sh_z(rx_z[1])]
shift_C = ["y", oa.sh_z(rx_z[2])]

# Trinocular positions
rx_x_tri = np.array([-519.62 * (3 / 2), 519.62 * (3 / 2), 0])
rx_z_tri = np.array([-300 * (3 / 2), -300 * (3 / 2), 600 * (3 / 2)])
el_tri = np.array(
    [oa.el_offset(rx_z_tri[0]), oa.el_offset(rx_z_tri[1]), oa.el_offset(rx_z_tri[2])]
)
az_tri = np.array(
    [oa.az_offset(rx_x_tri[0]), oa.az_offset(rx_x_tri[1]), oa.az_offset(rx_x_tri[2])]
)

shift_A_tri = ["xy", oa.sh_x(rx_x_tri[0]), oa.sh_z(rx_z_tri[0])]
shift_B_tri = ["xy", oa.sh_x(rx_x_tri[1]), oa.sh_z(rx_z_tri[1])]
shift_C_tri = ["y", oa.sh_z(rx_z_tri[2])]

def tele_geo_init(x, y, z, el, az):
    tele_geo = tg.initialize_telescope_geometry()
    tele_geo.rx_x = x
    tele_geo.rx_y = y
    tele_geo.rx_z = z
    tele_geo.el0 += el
    tele_geo.az0 += az
    return tele_geo

## Standard Deviation of Fitting Performance
Repeating the holography measurements and fitting, we determine the standard deviation of the HWFE.

In [ ]:
rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
tele_geo = tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])

th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)

rxmirror_C = af.ray_mirror_pts(rx3, tele_geo, th, ph)
singular_model = pickle.load(open("model_singular1.sav", "rb"))
initial = []
final = []
final_m1 = []
final_m2 = []
reps = 100
for ii in range(reps):
    # Define FOV of receiver feed (RX) positions
    # (i.e. define direction of outgoing rays from the RX).
    tele_geo = tg.initialize_telescope_geometry()

    # Define adjuster offsets, as random distribution on the micron scale
    adj_1 = np.random.randn(77 * 5) * 20  # [um]
    adj_2 = np.random.randn(69 * 5) * 25  # [um]

    # Define panels on M1 and M2. Here you can define the
    # magnitude of the adjuster offsets on each mirror:
    pan_mod_m2 = pm.panel_model_from_adjuster_offsets(
        2, adj_2, 1, 0
    )  # Panel Model on M2
    pan_mod_m1 = pm.panel_model_from_adjuster_offsets(
        1, adj_1, 1, 0
    )  # Panel Model on M1

    rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
    tele_geo = tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
    dat_C = afit.take_measurement(
        np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_C
    )
    dat_C = np.loadtxt(dat_C)
    x_C, y_C, meas_C, ampl_C, geo = afit.analyze_holography(
        dat_C, tele_geo, 0, 1, 0, shift_C
    )
    meas_C = np.where(
        (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, meas_C - np.mean(meas_C), 0
    )

    out3 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx3, tele_geo, th, ph, rxmirror_C
    )  # apert. fields

    beam3 = ff.far_field_sim(out3, tele_geo, rx3)  # far field beam
    amp3 = 20 * np.log10(
        abs(beam3[2, :]) / np.max(abs(beam3[2, :]))
    )  # far field beam amplitude [dB]

    np.savetxt(
        "/data/chesmore/sim_out/rx_" + str(rx3) + "_holog.txt",
        np.c_[
            np.real(beam3[0, :]),
            np.real(beam3[1, :]),
            np.real(beam3[2, :]),
            np.imag(beam3[2, :]),
        ],
    )

    rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
    tele_geo = tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
    dat_C = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx3) + "_holog.txt")
    x_C, y_C, phase_C, ampl_C, geo = afit.analyze_holography(
        dat_C, tele_geo, 0, 1, 0, shift_C
    )
    phase_C = np.where(
        (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, phase_C - np.mean(phase_C), 0
    )
    phase_C -= meas_C

    pathl_meas1 = np.reshape((phase_C), (1, len(phase_C)))
    adj_fit1 = singular_model.predict(pathl_meas1)
    adjs_real = np.concatenate((adj_1, adj_2)) / 1e3

    adjust = adj_fit1[0]

    rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
    tele_geo_C = tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
    phase_C = af.model_of_adj_offs(adjs_real * 1e3, shift_C, tele_geo_C, "total")
    # phase_fit_C1 = af.model_of_adj_offs(adjust*1e3, shift_C, tele_geo_C,'total')
    phase_tot_new_C1 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "total"
    )
    phase_m1_new_C1 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m1"
    )
    phase_m2_new_C1 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m2"
    )
    # phase_m1_model_C1 = af.model_of_adj_offs((adjust*1e3), shift_C, tele_geo_C,'m1')
    # phase_m2_model_C1 = af.model_of_adj_offs((adjust*1e3), shift_C, tele_geo_C,'m2')
    # phase_m1_C = af.model_of_adj_offs((adjs_real*1e3), shift_C, tele_geo_C,'m1')
    # phase_m2_C = af.model_of_adj_offs((adjs_real*1e3), shift_C, tele_geo_C,'m2')

    final_m1.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_m1_new_C1 - np.mean(phase_m1_new_C1) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )
    final_m2.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_m2_new_C1 - np.mean(phase_m2_new_C1) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )

    initial.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_C - np.mean(phase_C) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )
    final.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (
                phase_tot_new_C1
                - np.mean(phase_tot_new_C1)
                - (meas_C - np.mean(meas_C))
            )
            / tele_geo.k,
        )
    )

In [ ]:
print(r"M1 HWFE: %.2f +/- %.2f" % (np.mean(final_m1), np.std(final_m1)))
print(r"M2 HWFE: %.2f +/- %.2f" % (np.mean(final_m2), np.std(final_m2)))

In [ ]:
rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
rx2 = np.array([rx_x[1], 209.09, rx_z[1]])

th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)

tele_geo = tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
rxmirror_A = af.ray_mirror_pts(rx1, tele_geo, th, ph)
tele_geo = tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
rxmirror_B = af.ray_mirror_pts(rx2, tele_geo, th, ph)

binocular_model = pickle.load(open("model_binocular2.sav", "rb"))

# initial2 = []
# final2 = []
# final2_m1 = []
# final2_m2 = []

reps = 100 - 27
for ii in range(reps):
    # Define FOV of receiver feed (RX) positions
    # (i.e. define direction of outgoing rays from the RX).
    tele_geo = tg.initialize_telescope_geometry()

    # Define adjuster offsets, as random distribution on the micron scale
    adj_1 = np.random.randn(77 * 5) * 20  # [um]
    adj_2 = np.random.randn(69 * 5) * 25  # [um]

    # Define panels on M1 and M2. Here you can define the
    # magnitude of the adjuster offsets on each mirror:
    pan_mod_m2 = pm.panel_model_from_adjuster_offsets(
        2, adj_2, 1, 0
    )  # Panel Model on M2
    pan_mod_m1 = pm.panel_model_from_adjuster_offsets(
        1, adj_1, 1, 0
    )  # Panel Model on M1

    rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
    tele_geo = tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
    dat_A = afit.take_measurement(
        np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_A
    )
    dat_A = np.loadtxt(dat_A)
    x_A, y_A, meas_A, ampl_A, geo = afit.analyze_holography(
        dat_A, tele_geo, 0, 1, 0, shift_A
    )
    meas_A = np.where(
        (abs(ampl_A) / np.max(abs(ampl_A))) >= 0.3, meas_A - np.mean(meas_A), 0
    )

    out1 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx1, tele_geo, th, ph, rxmirror_A
    )  # apert. fields

    beam1 = ff.far_field_sim(out1, tele_geo, rx1)  # far field beam

    rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
    tele_geo = tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
    dat_B = afit.take_measurement(
        np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_B
    )
    dat_B = np.loadtxt(dat_B)
    x_B, y_B, meas_B, ampl_B, geo = afit.analyze_holography(
        dat_B, tele_geo, 0, 1, 0, shift_B
    )
    meas_B = np.where(
        (abs(ampl_B) / np.max(abs(ampl_B))) >= 0.3, meas_B - np.mean(meas_B), 0
    )

    out2 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx2, tele_geo, th, ph, rxmirror_B
    )  # apert. fields

    beam2 = ff.far_field_sim(out2, tele_geo, rx2)  # far field beam

    np.savetxt(
        "/data/chesmore/sim_out/rx_" + str(rx1) + "_holog.txt",
        np.c_[
            np.real(beam1[0, :]),
            np.real(beam1[1, :]),
            np.real(beam1[2, :]),
            np.imag(beam1[2, :]),
        ],
    )
    np.savetxt(
        "/data/chesmore/sim_out/rx_" + str(rx2) + "_holog.txt",
        np.c_[
            np.real(beam2[0, :]),
            np.real(beam2[1, :]),
            np.real(beam2[2, :]),
            np.imag(beam2[2, :]),
        ],
    )

    rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
    tele_geo = tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
    dat_A = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx1) + "_holog.txt")
    x_A, y_A, phase_A, ampl_A, geo = afit.analyze_holography(
        dat_A, tele_geo, 0, 1, 0, shift_A
    )

    rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
    tele_geo = tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
    dat_B = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx2) + "_holog.txt")
    x_B, y_B, phase_B, ampl_B, geo = afit.analyze_holography(
        dat_B, tele_geo, 0, 1, 0, shift_B
    )

    phase_A = np.where(
        (abs(ampl_A) / np.max(abs(ampl_A))) >= 0.3, phase_A - np.mean(phase_A), 0
    )
    phase_B = np.where(
        (abs(ampl_B) / np.max(abs(ampl_B))) >= 0.3, phase_B - np.mean(phase_B), 0
    )
    phase_A -= meas_A
    phase_B -= meas_B

    pathl_meas2 = np.reshape(
        (np.concatenate((phase_A, phase_B))),
        (1, len(np.concatenate((phase_A, phase_B)))),
    )
    adj_fit2 = binocular_model.predict(pathl_meas2)
    adjs_real = np.concatenate((adj_1, adj_2)) / 1e3

    adjust = adj_fit2[0]

    rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
    tele_geo_C = tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
    phase_C = af.model_of_adj_offs(adjs_real * 1e3, shift_C, tele_geo_C, "total")
    phase_tot_new_C2 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "total"
    )
    phase_m1_new_C2 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m1"
    )
    phase_m2_new_C2 = af.model_of_adj_offs(
        ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m2"
    )

    final2_m1.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_m1_new_C2 - np.mean(phase_m1_new_C2) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )
    final2_m2.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_m2_new_C2 - np.mean(phase_m2_new_C2) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )

    initial2.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (phase_C - np.mean(phase_C) - (meas_C - np.mean(meas_C)))
            / tele_geo.k,
        )
    )
    final2.append(
        oa.rms(
            x_C,
            y_C,
            1e6
            * (
                phase_tot_new_C2
                - np.mean(phase_tot_new_C2)
                - (meas_C - np.mean(meas_C))
            )
            / tele_geo.k,
        )
    )

In [ ]:
print(np.mean(final2), np.std(np.array(initial2) - np.array(final2)))
print(r"M1 HWFE: %.2f $\pm$ %.2f" % (np.mean(final2_m1), np.std(final2_m1)))
print(r"M2 HWFE: %.2f $\pm$ %.2f" % (np.mean(final2_m2), np.std(final2_m2)))